In [1]:
from kashgari.corpus import SMP2018ECDTCorpus
import kashgari
from kashgari.tasks.classification import BiLSTM_Model, BiGRU_Model
from kashgari.embeddings import BERTEmbedding

import logging
logging.basicConfig(level='DEBUG')

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

In [3]:
train_x, train_y = SMP2018ECDTCorpus.load_data('train')
test_x, test_y = SMP2018ECDTCorpus.load_data('test')
valid_x, valid_y = SMP2018ECDTCorpus.load_data('valid')

DEBUG:root:loaded 1881 samples from C:\Users\Administrator\.kashgari\datasets\SMP2018ECDTCorpus\train.csv. Sample:
x[0]: ['你', '喜', '欢', '有', '钱', '人', '吗']
y[0]: chat
DEBUG:root:loaded 770 samples from C:\Users\Administrator\.kashgari\datasets\SMP2018ECDTCorpus\test.csv. Sample:
x[0]: ['第', '二', '梦']
y[0]: video
DEBUG:root:loaded 418 samples from C:\Users\Administrator\.kashgari\datasets\SMP2018ECDTCorpus\valid.csv. Sample:
x[0]: ['哦', '哦', '，', '查', '一', '下', '香', '港', '最', '新', '上', '映', '电', '影', '。']
y[0]: cinemas


In [4]:
early_stop = EarlyStopping('val_acc')

# 训练分类模型

In [16]:
model = BiLSTM_Model()
model.fit(train_x, train_y, valid_x, valid_y, epochs=100, callbacks=[early_stop])

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 15)]              0         
_________________________________________________________________
layer_embedding (Embedding)  (None, 15, 100)           87500     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               234496    
_________________________________________________________________
dense_1 (Dense)              (None, 31)                7967      
Total params: 329,963
Trainable params: 329,963
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
30/30 [==============================] - 5s 169ms/step - loss: 3.1124 - acc: 0.1951 - val_loss: 2.8804 - val_acc: 0.1962
Epoch 2/100
30/30 [==============================] - 2s 55ms/step - loss: 2.5966 - acc: 0.2472 - val_loss: 2.6741 - va

In [27]:
model.predict(['你说说看呢'])

['chat']

# 使用预训练语言模型进行迁移学习

In [5]:
bert_weights_path = r"D:\Github\NLP\Projects\data\bert_pretrained_model\chinese_L-12_H-768_A-12"

In [10]:
bert_embedding = BERTEmbedding(bert_weights_path, task=kashgari.CLASSIFICATION, sequence_length=100)

In [11]:
model = BiGRU_Model(bert_embedding)
model.fit(train_x, train_y, valid_x, valid_y)

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 100)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 100)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 100, 768), ( 16226304    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 100, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

KeyboardInterrupt: 

# 调整模型参数

In [45]:
hyper = BiLSTM_Model.get_default_hyper_parameters()
hyper

{'layer_bi_lstm': {'units': 128, 'return_sequences': False},
 'layer_dense': {'activation': 'softmax'}}

In [46]:
hyper['layer_bi_lstm']['units'] = 32
model = BiLSTM_Model(hyper_parameters=hyper)

In [55]:
model.hyper_parameters

{'layer_bi_lstm': {'units': 32, 'return_sequences': False},
 'layer_dense': {'activation': 'softmax'}}